In [1]:
print("hello")

hello


In [2]:
from datasets import load_dataset

# Load Dataset
dataset_name = "glue"
task_name = "sst2"
dataset = load_dataset(dataset_name, task_name, split="train")
validation_set = load_dataset('glue', 'sst2', split='validation')

Found cached dataset glue (/mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
def create_label_text(label):
    label_map = {
        0 : 'negative',
        1 : 'positive',
    }
    
    return {'label_text': label_map[label]}

In [4]:
# Convert digital labels to text labels
dataset = dataset.map(create_label_text, input_columns=['label'])
validation_set = validation_set.map(create_label_text, input_columns=['label'])

Loading cached processed dataset at /mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ad792f34c7b5c010.arrow
Loading cached processed dataset at /mnt/nlp4sd/hugging_face_cache/ysong/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-bb9884afcd56c7c4.arrow


In [5]:
dataset[0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0,
 'label_text': 'negative'}

In [6]:
validation_set[0]

{'sentence': "it 's a charming and often affecting journey . ",
 'label': 1,
 'idx': 0,
 'label_text': 'positive'}

In [7]:
def convert_into_prompt_template(system_prompt, user_message, train=True, label_text=""):
    """
    Convert the dataset into the prompt template format as follows:
    <s>[INST] <<SYS>>
    {{ system_prompt }}
    <</SYS>>

    {{ user_message }} [/INST] 
    Sentiment: {{ label }} </s>      ## label is neccessary for training, but not for inference
    """
    if train:
        text = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\nSentence: {user_message} [/INST]\nSentiment: {label_text} </s>"
    else:
        text = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\nSentence: {user_message} [/INST]\nSentiment: "

    return text

In [8]:
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: 'positive' or 'negative'."

### Simple test (Useless Right Now)

In [9]:
user_msg_2 = dataset[0]["sentence"]
label_2 = dataset[0]["label_text"]
converted1 = convert_into_prompt_template(system_prompt, user_msg_2, False, label_2)
print(converted1)

<s>[INST] <<SYS>>
You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: 'positive' or 'negative'.
<</SYS>>

Sentence: hide new secretions from the parental units  [/INST]
Sentiment: 


### Continue

In [35]:
def map_dataset(system_prompt, dataset, train=True):
    # Convert the dataset into the format required by the model
    def convert(sentence, label_text):
        converted_inputs = convert_into_prompt_template(system_prompt, sentence, train, label_text)
        return {'text': converted_inputs, 'label_text': label_text}

    return dataset.map(convert, input_columns=['sentence', 'label_text'], batched=False, remove_columns=['sentence', 'label', 'idx', 'label_text'])

In [36]:
new_dataset = map_dataset(system_prompt, dataset, train=True)
new_validation_set = map_dataset(system_prompt, validation_set, train=False)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [37]:
new_dataset[0]

{'label_text': 'negative',
 'text': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: 'positive' or 'negative'.\n<</SYS>>\n\nSentence: hide new secretions from the parental units  [/INST]\nSentiment: negative </s>"}

In [38]:
new_validation_set[0]

{'label_text': 'positive',
 'text': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: 'positive' or 'negative'.\n<</SYS>>\n\nSentence: it 's a charming and often affecting journey .  [/INST]\nSentiment: "}

### Merge datasets

In [39]:
from datasets import DatasetDict

dataset_to_upload = DatasetDict({
    'train': new_dataset,
    'validation': new_validation_set
})

In [40]:
dataset_to_upload

DatasetDict({
    train: Dataset({
        features: ['label_text', 'text'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['label_text', 'text'],
        num_rows: 872
    })
})

### Upload dataset

In [41]:
dataset_to_upload.push_to_hub("OneFly7/llama2-sst2-fine-tuning")

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/68 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split validation to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]